# Intensity

This notebook is similiar to the second part of the `monte_carlo` notebook, but it runs everything for only one price. The time between adjustments of prices gets split into `B` parts and for each purchase the time slot `o` fit is tracked. At the end, it outputs a probability for each time slot, which corresponds to how likely a purchase is in this time slot after a price adjustment. The probabilities are saved in the `cond_prob_out` file.

In [ ]:
import numpy as np
import random
import csv

In [ ]:
numberPlayers = 5
timeHorizon = 1
numberMarketReactionEvents = round(1.1*timeHorizon)
meanTimeBetweenCustomerArrival = 0.05
numberOfCustomerEvents = 20
prices = [6.5]
watchedCompany = 0
numberSimulations = 10000

In [ ]:
pricesCompanies = np.array([round(random.uniform(6,16),2) for _ in range(numberPlayers)])
qualityCompanies = np.array([round(random.uniform(0.5,5.5)) for _ in range(numberPlayers)])
ratingCompanies = np.array([round(random.uniform(90,100),1) for _ in range(numberPlayers)])

In [ ]:
numberConsideredMarketSituations = 1

pricesCompaniesPred = np.array([[round(random.uniform(6,16),2) for _ in range(numberConsideredMarketSituations)]  for _ in range(numberPlayers)])
qualityCompaniesPred = np.array([[round(random.uniform(0.5,5.5)) for _ in range(numberConsideredMarketSituations)]  for _ in range(numberPlayers)])
ratingCompaniesPred = np.array([[round(random.uniform(90,100),1) for _ in range(numberConsideredMarketSituations)] for _ in range(numberPlayers)])

In [ ]:
def calculateTimeOfAdjustmentsCompany(company, simulationNumber, i, timeBetweenMarketEventsCompany):
    if i==0:
        if company != watchedCompany:
            return(random.uniform(0,1))
        else:
            return(reactionTimeCompany)
    else:
        return(calculateTimeOfAdjustmentsCompany(company, simulationNumber, i-1, timeBetweenMarketEventsCompany)+ timeBetweenMarketEventsCompany[simulationNumber][company][i])
    
def calculateTimeOfCustomerArrival(customer, simulationNumber, timeBetweenMarketEventsCustomer):
    if customer==0:
        return 0
    else:
        return(calculateTimeOfCustomerArrival(customer-1, simulationNumber, timeBetweenMarketEventsCustomer) + timeBetweenMarketEventsCustomer[simulationNumber][customer])
    
def priceAdjustmentRules(c, pricesCompanies):
    return {
        '0' : pricesCompanies[0],
        '1' : 9 if pricesCompanies[c]<4 else round(min([max(3.9, pricesCompanies[company]-0.5) for company in range(numberPlayers) if company != c]),2),
        '2' : 12 if pricesCompanies[c]<6 else round(min([max(5.9, pricesCompanies[company]-0.5) for company in range(numberPlayers) if company != c]),2),
        '3' : round(random.uniform(11,11),2),
        '4' : round(random.uniform(13,13),2)
    }[str(c)]

def calculatePrices(pricesCompaniesAdjustments, adjustingFirm):
    for i in range(len(pricesCompaniesAdjustments)):
        if i > 0:
            pricesCompaniesAdjustments[i, adjustingFirm[i]] = priceAdjustmentRules(adjustingFirm[i],pricesCompaniesAdjustments[i-1])
    return pricesCompaniesAdjustments

In [ ]:
m=2
numberCustomerEvents=4
simulatedHorizon = 1
reactionTimeCompany = timeHorizon

timeBetweenMarketEventsCompany = np.array([[[random.uniform(0.8,1.2) if company!= watchedCompany else reactionTimeCompany for i in range(m)] for company in range(numberPlayers)] for _ in range(numberSimulations)])
timeOfAdjustmentsCompany = np.array([[[calculateTimeOfAdjustmentsCompany(company,simulationNumber ,i,timeBetweenMarketEventsCompany) for i in range(m)] for company in range(numberPlayers)] for simulationNumber in range(numberSimulations)])

priceAdjustmentsBeforeHorizon = np.array([set([timeOfAdjustmentsCompany[simulationNumber].flatten()[i] for i in range(len(timeOfAdjustmentsCompany[simulationNumber].flatten())) if timeOfAdjustmentsCompany[simulationNumber].flatten()[i] < timeHorizon])for simulationNumber in range(numberSimulations)])
priceAdjustmentsBeforeHorizon = np.array([sorted(priceAdjustmentsBeforeHorizon[simulationNumber]) for simulationNumber in range(numberSimulations)])

numberPriceAdjustmentsBeforeHorizon = np.array([len(priceAdjustmentsBeforeHorizon[simulationNumber]) for simulationNumber in range(numberSimulations)])

timeMarketEvents = np.array([[0 if i==0 else priceAdjustmentsBeforeHorizon[simulationNumber][i-1] for i in range(numberPriceAdjustmentsBeforeHorizon[simulationNumber]+1)]for simulationNumber in range(numberSimulations)])
timeBetweenMarketEventsCustomer = np.array([[timeMarketEvents[simulationNumber][i+1]-timeMarketEvents[simulationNumber][i] for i in range(numberPriceAdjustmentsBeforeHorizon[simulationNumber])] for simulationNumber in range(numberSimulations)])

timeBetweenCustomerEvents = np.array([[meanTimeBetweenCustomerArrival*np.random.exponential() for _ in range(numberCustomerEvents)]for simulationNumber in range(numberSimulations)])
timeCustomerEvents = np.array([[calculateTimeOfCustomerArrival(customer, simulationNumber, timeBetweenMarketEventsCustomer) for customer in range(numberCustomerEvents)] for simulationNumber in range(numberSimulations)])

adjustingFirm = np.array([[min(np.where(timeOfAdjustmentsCompany[simulationNumber]==priceAdjustmentsBeforeHorizon[simulationNumber][i] )[0]) for i in range(numberPriceAdjustmentsBeforeHorizon[simulationNumber])] for simulationNumber in range(numberSimulations)])

randomCustomerScoring = np.array([[0 for _ in range(numberPlayers)] for simulationNumber in range(numberSimulations)])
randomCustomerDecision = np.array([[-1 for _ in range(numberCustomerEvents)] for simulationNumber in range(numberSimulations)])
randomCustomerScoringQuality = np.array([[random.uniform(0,1) for _ in range(numberCustomerEvents)] for simulationNumber in range(numberSimulations)])
randomCustomerScoringRating = np.array([[random.uniform(0,0.5) for _ in range(numberCustomerEvents)] for simulationNumber in range(numberSimulations)])

As = np.array([[[0 for _ in range(numberConsideredMarketSituations)]for _ in range(numberSimulations)] for simulationNumber in range(numberPlayers)])
A = np.array([[[float(0) for _ in range(numberConsideredMarketSituations)]for _ in range(6)]for _ in range(numberPlayers)])
bb = 20
At = np.array([[[float(0) for _ in range(numberConsideredMarketSituations)]for _ in range(reactionTimeCompany*bb+1)]for _ in range(numberPlayers)])

P_ori = np.array([[float(0) for _ in range(len(prices))] for _ in range(numberConsideredMarketSituations)])
P_ori2 = np.array([[float(0) for _ in range(len(prices))] for _ in range(numberConsideredMarketSituations)])

playersWithoutWatched = [i for i in range(numberPlayers) if i!=watchedCompany]

for w in range(numberConsideredMarketSituations):
    pricesCompaniesPredW= pricesCompaniesPred[:,w]
    qualityCompanies = [2+0*qualityCompaniesPred[k, w] for k in range(numberPlayers)]
    ratingCompanies = [98+0*ratingCompaniesPred[k, w] for k in range(numberPlayers)]
    customerScoringQualityRating = np.array([[[(randomCustomerScoringQuality[s][c]*qualityCompanies[k] +
                                                          randomCustomerScoringRating[s][c]*(100-ratingCompanies[k]))for k in range(numberPlayers)] for c in range(numberCustomerEvents)] for s in range(numberSimulations)])
    for a in range(len(prices)):
        AsSim = np.array(As[:,:,w])
        randomCustomerDecision = np.array([[-1 for _ in range(numberCustomerEvents)] for simulationNumber in range(numberSimulations)])
        for s in range(numberSimulations):
            randomCustomerScoringSim = randomCustomerScoring[s]
            pricesCompanies[watchedCompany]=prices[a]
            if numberPlayers> 1:
                pricesCompanies[playersWithoutWatched]=[prices[a]+0.5+2*((i+1)-2)+0*pricesCompaniesPredW[i] for i in playersWithoutWatched]
            numberPriceAdjustmentsBeforeHorizonCurrent = numberPriceAdjustmentsBeforeHorizon[s]
            pricesCompaniesAdjustments = np.array([[pricesCompanies[i] for i in range(numberPlayers)]for _ in range(numberPriceAdjustmentsBeforeHorizonCurrent)])
            pricesCompaniesAdjustments = np.array(calculatePrices(pricesCompaniesAdjustments, adjustingFirm[s]))
            for i in range(numberPriceAdjustmentsBeforeHorizonCurrent):
                gen = list((x for x in range(numberCustomerEvents) if timeMarketEvents[s][i]< timeCustomerEvents[s][x]< timeMarketEvents[s][i+1]))
                randomCustomerScoringSim = np.array([[(pricesCompaniesAdjustments[i][k] + customerScoringQualityRating[s,c,k])for k in range(numberPlayers)] for c in gen])
                indexGen = [x - min(gen) for x in gen]
                for c in indexGen:
                    if min(randomCustomerScoringSim[c]) < random.uniform(5,15):
                        randomCustomerDecision[s][c] = np.argmin([randomCustomerScoringSim[c][company] for company in range(numberPlayers)])
                        AsSim[randomCustomerDecision[s][c]][s]=AsSim[randomCustomerDecision[s][c]][s] +1 
                        b = int(round((timeCustomerEvents[s][c]+1/bb/2)*bb))
                        At[randomCustomerDecision[s][c]][b][w]=At[randomCustomerDecision[s][c]][b][w]+1
            for x in range(6):
                for company in range(numberPlayers):
                    salesForDecision = np.array(AsSim[company,:])
                    A[company][x][w] = 1/numberSimulations*len(salesForDecision[salesForDecision==x])
            P_ori[w][a] = 1-A[watchedCompany][0][w]
            P_ori2[w][a] = A[watchedCompany][2][w]

with open('cond_prob_out_S3.csv', 'w') as file:
    writer = csv.writer(file, delimiter='\t')
    for w in range(numberConsideredMarketSituations):
        for t in range(bb+1):
            line = [t/bb, At[watchedCompany][t][w]/numberSimulations]
            writer.writerow(line)
            